# Counter

In this notebook the idea is to count values in tweet texts, to calculate tweets length and word count and to create ngrams to find out which words most often occur together.

# Importing libraries

In [1]:
# Import Libraries

from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

# Reading and eyeballing dataset

In [2]:
data = pd.read_csv('data_new.csv')
data.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Tweet Text,Followers,Follows,Retweets,Favorites,Location,Airport_Twitter_Tag,Airport_IATA_Codes,Airport_Names,tokens,no_tokens,vader_score,vader_sentiment_label
0,0,0,15421,2017-04-07,update flt departed back dublin but flt offloa...,20959.0,240.0,0.0,0.0,"Belfast, Northern Ireland",@dublinairport,DUB,Dublin Airport,"['update', 'flt', 'departed', 'back', 'dublin'...","['update', 'flt', 'departed', 'back', 'dublin'...",0.0000,0
1,1,1,25645,2018-12-03,lads come rally the troops for your pals they ...,1433.0,637.0,0.0,0.0,"Dublin, Ireland",@dublinairport,DUB,Dublin Airport,"['lads', 'come', 'rally', 'troops', 'pals', 'a...","['lads', 'come', 'rally', 'troops', 'pals', 'a...",0.0000,0
2,3,3,181260,2020-06-12,june british airways resumed daily service fro...,51130.0,3181.0,0.0,0.0,San Francisco Int'l Airport,@heathrowairport,LHR,Heathrow Airport,"['june', 'british', 'airways', 'resumed', 'dai...","['june', 'british', 'airways', 'resumed', 'dai...",0.0000,0
3,5,5,274014,2016-11-07,amazing trip now way home with thx jfk for the...,90.0,318.0,0.0,0.0,NaN,@heathrowairport,LHR,Heathrow Airport,"['amazing', 'trip', 'way', 'home', 'thx', 'jfk...","['amazing', 'trip', 'way', 'home', 'thx', 'jfk...",0.8860,1
4,6,6,274016,2016-11-07,from brazil get hours there and have nice flig...,1610.0,329.0,0.0,0.0,"Rio de Janeiro, Brasil",@heathrowairport,LHR,Heathrow Airport,"['brazil', 'get', 'hours', 'nice', 'flight', '...","['brazil', 'get', 'hours', 'nice', 'flight', '...",-0.0258,0
5,7,7,274022,2016-11-07,fast track yeah right slow,93.0,219.0,0.0,0.0,Ireland,@dublinairport,DUB,Dublin Airport,"['fast', 'track', 'yeah', 'right', 'slow']","['fast', 'track', 'yeah', 'right', 'slow']",0.2960,1
6,8,8,274023,2016-11-07,what with drop off lanes terminal the queue re...,358.0,379.0,0.0,0.0,London - Strüer - OnAir,@heathrowairport,LHR,Heathrow Airport,"['drop', 'lanes', 'terminal', 'queue', 'reache...","['drop', 'lanes', 'terminal', 'queue', 'reache...",-0.2263,0
7,9,9,274024,2016-11-07,landan,195.0,1118.0,0.0,0.0,NaN,@gatwick_airport,LGW,Gatwick Airport LGW,['landan'],['landan'],0.0000,0
8,10,10,274027,2016-11-07,beautiful copenhagen,48.0,199.0,0.0,0.0,"Isle of Wight, United Kingdom",@gatwick_airport,LGW,Gatwick Airport LGW,"['beautiful', 'copenhagen']","['beautiful', 'copenhagen']",0.5994,1
9,11,11,274028,2016-11-07,you need sort out your doors arrivals the nort...,134.0,792.0,0.0,0.0,west sussex,@gatwick_airport,LGW,Gatwick Airport LGW,"['need', 'sort', 'doors', 'arrivals', 'north',...","['need', 'sort', 'doors', 'arrivals', 'north',...",0.0762,1


# Creating a column for string

In [3]:
# Luon uuden muuttujan 'vader_sentiment_label_string'
data['vader_sentiment_label_string'] = data['vader_sentiment_label'].replace({1 : 'Positive', 0 : 'Negative'})
data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Tweet Text,Followers,Follows,Retweets,Favorites,Location,Airport_Twitter_Tag,Airport_IATA_Codes,Airport_Names,tokens,no_tokens,vader_score,vader_sentiment_label,vader_sentiment_label_string
0,0,0,15421,2017-04-07,update flt departed back dublin but flt offloa...,20959.0,240.0,0.0,0.0,"Belfast, Northern Ireland",@dublinairport,DUB,Dublin Airport,"['update', 'flt', 'departed', 'back', 'dublin'...","['update', 'flt', 'departed', 'back', 'dublin'...",0.0000,0,Negative
1,1,1,25645,2018-12-03,lads come rally the troops for your pals they ...,1433.0,637.0,0.0,0.0,"Dublin, Ireland",@dublinairport,DUB,Dublin Airport,"['lads', 'come', 'rally', 'troops', 'pals', 'a...","['lads', 'come', 'rally', 'troops', 'pals', 'a...",0.0000,0,Negative
2,3,3,181260,2020-06-12,june british airways resumed daily service fro...,51130.0,3181.0,0.0,0.0,San Francisco Int'l Airport,@heathrowairport,LHR,Heathrow Airport,"['june', 'british', 'airways', 'resumed', 'dai...","['june', 'british', 'airways', 'resumed', 'dai...",0.0000,0,Negative
3,5,5,274014,2016-11-07,amazing trip now way home with thx jfk for the...,90.0,318.0,0.0,0.0,NaN,@heathrowairport,LHR,Heathrow Airport,"['amazing', 'trip', 'way', 'home', 'thx', 'jfk...","['amazing', 'trip', 'way', 'home', 'thx', 'jfk...",0.8860,1,Positive
4,6,6,274016,2016-11-07,from brazil get hours there and have nice flig...,1610.0,329.0,0.0,0.0,"Rio de Janeiro, Brasil",@heathrowairport,LHR,Heathrow Airport,"['brazil', 'get', 'hours', 'nice', 'flight', '...","['brazil', 'get', 'hours', 'nice', 'flight', '...",-0.0258,0,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240799,273183,273183,868309,2021-03-23,thank you,11378.0,1057.0,0.0,0.0,"Manchester, England",@stn_airport,STN,London Stansted Airpor,['thank'],['thank'],0.3612,1,Positive
240800,273184,273184,868344,2021-03-24,good morning stansted airport,95.0,958.0,0.0,0.0,NaN,@stn_airport,STN,London Stansted Airpor,"['good', 'morning', 'stansted', 'airport']","['good', 'morning', 'stansted', 'airport']",0.4404,1,Positive
240801,273185,273185,868346,2021-03-24,omg only you knew how much missed you had litt...,95.0,958.0,0.0,1.0,NaN,@stn_airport,STN,London Stansted Airpor,"['omg', 'knew', 'much', 'missed', 'little', 'h...","['omg', 'knew', 'much', 'missed', 'little', 'h...",0.0525,1,Positive
240802,273186,273186,868347,2021-03-24,spain blood literally dad half spanish family ...,95.0,958.0,0.0,1.0,NaN,@stn_airport,STN,London Stansted Airpor,"['spain', 'blood', 'literally', 'dad', 'half',...","['spain', 'blood', 'literally', 'dad', 'half',...",0.0000,0,Negative


# Calculating tweet's length and counting values

In [4]:
#Calculating tweet's length and word count
data['text_len'] = data['Tweet Text'].astype(str).apply(len)
data['text_word_count'] = data['Tweet Text'].apply(lambda x: len(str(x).split()))

In [5]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [6]:
#Count_values for sentiment
count_values_in_column(data,"vader_sentiment_label_string")

,Total,Percentage
Negative,133640,55.5
Positive,107164,44.5


In [7]:
# looks like no big difference between tweet's length across negative and positive sentiments
round(pd.DataFrame(data.groupby("vader_sentiment_label_string").text_len.mean()),2)

,text_len
vader_sentiment_label_string,
Negative,49.32
Positive,53.97


In [8]:
round(pd.DataFrame(data.groupby("vader_sentiment_label_string").text_word_count.mean()),2)

,text_word_count
vader_sentiment_label_string,
Negative,8.26
Positive,9.15


# Function for ngrams

In [9]:
#Function to ngram
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [10]:
#n2_bigram
# which words most often occur together in tweets
# interesting to see that car park, hand luggage, flight delayed, security staff, baggage reclaim occur together
n2_bigrams = get_top_n_gram(data['Tweet Text'],(2,2),50)

n2_bigrams

[('good morning', 2008),
 ('car park', 1916),
 ('passport control', 1736),
 ('dublin airport', 1130),
 ('north terminal', 1104),
 ('saga premium', 1047),
 ('fast track', 988),
 ('new york', 946),
 ('hand luggage', 912),
 ('fingers crossed', 859),
 ('manchester airport', 844),
 ('heathrow airport', 801),
 ('south terminal', 791),
 ('times week', 728),
 ('duty free', 705),
 ('times day', 693),
 ('looking forward', 683),
 ('customer service', 678),
 ('case number', 677),
 ('apology accepted', 670),
 ('nearly time', 668),
 ('accepted nearly', 664),
 ('time create', 664),
 ('create case', 664),
 ('number happens', 664),
 ('love win', 577),
 ('thanks reply', 546),
 ('long stay', 543),
 ('flight delayed', 529),
 ('let know', 525),
 ('addis ababa', 512),
 ('looks like', 504),
 ('morning heathrow', 496),
 ('security staff', 486),
 ('tomorrow morning', 468),
 ('merry christmas', 454),
 ('official complaint', 454),
 ('baggage reclaim', 452),
 ('gatwick airport', 448),
 ('filed official', 440),
 (

In [11]:
#n3_trigram
# which words most often occur together in tweets
# interesting to see that stay car park, fast track security, hand luggage, queue passport control, flight delayed hours, long stay parking, security hand luggage occur together
n3_trigrams = get_top_n_gram(data['Tweet Text'],(3,3),50)

n3_trigrams

[('apology accepted nearly', 664),
 ('accepted nearly time', 664),
 ('nearly time create', 664),
 ('time create case', 664),
 ('create case number', 664),
 ('case number happens', 664),
 ('filed official complaint', 438),
 ('official complaint agents', 438),
 ('complaint agents lane', 438),
 ('agents lane dangerous', 438),
 ('lane dangerous rac', 438),
 ('good morning heathrow', 370),
 ('stay car park', 352),
 ('fast track security', 222),
 ('long stay car', 204),
 ('happy new year', 185),
 ('short stay car', 155),
 ('landed hour ago', 154),
 ('queue passport control', 130),
 ('golden gate strait', 127),
 ('queues passport control', 124),
 ('penguin giraffe elephant', 115),
 ('meet greet parking', 104),
 ('dream come true', 99),
 ('angeles international airport', 98),
 ('los angeles international', 97),
 ('thanks quick reply', 93),
 ('fairytale new york', 92),
 ('saga premium class', 90),
 ('terminal car park', 89),
 ('flight delayed hours', 89),
 ('thanks quick response', 89),
 ('havi